入門ウェブハンドリングp176

$$
\sigma_r
$$

In [18]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from numba import njit

In [19]:
"""パラメータ"""
R_C = 50  # コア半径 [mm]
R_CIN = 40  # コア内径 [mm]
E_CIN = 1  # コアヤング率
NU_C = 1

N = 30000  # 巻数
NU = 1  # 半径方向ポアソン比
#T = 0.002  # シート厚み[mm]

# ポリプロピレン
K1 = 0.1e-3  # [GPa]
K2 = 260.0e-3  # [GPa]
T = 25.4e-3 # ウェブ厚み[mm]
E_THETA = 1.65  # 接線方向ヤング率[GPa]

E_C = E_CIN / (((R_C/R_CIN)**2+1) / ((R_C/R_CIN)**2-1) - NU_C)


T = 100 # 巻取り張力[N]

r = np.linspace(R_C, R_C+T*N, N)  # 半径リスト

In [23]:
#@njit
def main():
    sigma_r = np.zeros(N)
    a = np.zeros(N)
    b = np.zeros(N)
    c = np.zeros(N)
    g = np.zeros(N)
    sigma_r_total = 0
    
    a = 1 + (3-NU) * T / (2 * r)
    c = 1 - (3-NU)/2 * T/r
    
    for n in range(N):
        sigma_r_total = 0
        if n == 0:
            a0 = 1
            b0 = (1 - E_THETA/E_C - NU) * T / R_C - 1
            sigma_r_1 = - T / (R_C + T)
            sigma_r_0 = - (b0 / a0) * sigma_r_1
            _sigma_r = sigma_r_0 + sigma_r_1
            e_r = K2 * (_sigma_r + K2)
            g[1] = np.sqrt(E_THETA / e_r)
        else:
            A = np.zeros((n, n))
            B = np.zeros((n, 1))
            
            for i in range(n):
                sigma_r_total += sigma_r[i]
                _e_r = K2 * (sigma_r_total + K2)
                _gg = E_THETA / _e_r
                b[i] = (1 + NU - _gg) * (T/r[i])**2 - 2
            
            for i in range(n):
                for j in range(n):
                    